<a href="https://colab.research.google.com/github/choycoy/final_project/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")

In [3]:
q_df['question_id'] = q_df['question_id'].str.replace('q', '')
q_df

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,1,b1,e1,b,1,1;2;179;181,1558093217098
1,2,b2,e2,a,1,15;2;182,1558093219720
2,3,b3,e3,b,1,14;2;179;183,1558093222784
3,4,b4,e4,b,1,9;2;179;184,1558093225357
4,5,b5,e5,c,1,8;2;179;181,1558093228439
...,...,...,...,...,...,...,...
13164,18139,b12202,e12202,b,2,24;26;183;182,1571733814684
13165,18140,b12203,e12203,a,2,24;33;183;182,1571733815331
13166,18141,b12204,e12204,a,2,24;26;183;182,1571733815951
13167,18142,b12205,e12205,a,2,24;26;183;182,1571733816585


In [4]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")




In [45]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10])

In [46]:
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)


             timestamp  solving_id  question_id user_answer  elapsed_time  \
0        1565096190868           1         5012           b         38000   
1        1565096221062           2         4706           c         24000   
2        1565096293432           3         4366           b         68000   
3        1565096339668           4         4829           a         42000   
4        1565096401774           5         6528           b         59000   
...                ...         ...          ...         ...           ...   
2999995  1567427902612        3156        10020           b         20000   
2999996  1567428015542        3157        17192           a         14000   
2999997  1567428333271        3158         5418           b         15000   
2999998  1567428412553        3159        10020           c          4000   
2999999  1567431236639        3160         6366           c         31000   

         user_id correct_answer  is_correct  
0              1             

In [47]:
# Assuming you have the dataframe df containing student interactions with columns: timestamp, solving_id, question_id, user_answer, elapsed_time, user_id, and is_correct (1 for correct, 0 for incorrect).

# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction

# Calculate average, minimum, and maximum elapsed time for each user
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')
merged_df['min_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('min')
merged_df['max_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('max')

# Drop unnecessary columns if needed
merged_df.drop(['correct_count', 'interaction_count'], axis=1, inplace=True)


0            0
1            1
2            2
3            2
4            2
          ... 
2994680    503
2994681    503
2994682    504
2994683    504
2994684    504
Name: correct_count, Length: 3000000, dtype: int64
0            1
1            2
2            3
3            4
4            5
          ... 
2994680    601
2994681    602
2994682    603
2994683    604
2994684    605
Name: interaction_count, Length: 3000000, dtype: int64


In [15]:
merged_df['next_is_correct'] = merged_df['is_correct'].shift(-1)

In [16]:
merged_df

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id,correct_answer,is_correct,correctness_rate,average_elapsed_time,min_elapsed_time,max_elapsed_time,next_is_correct
0,1565096190868,1,5012,b,38000,1,c,0,0.000000,49578.441774,666,1200000,1.0
1,1565096221062,2,4706,c,24000,1,c,1,0.500000,49578.441774,666,1200000,1.0
2,1565096293432,3,4366,b,68000,1,b,1,0.666667,49578.441774,666,1200000,0.0
3,1565096339668,4,4829,a,42000,1,c,0,0.500000,49578.441774,666,1200000,0.0
4,1565096401774,5,6528,b,59000,1,d,0,0.400000,49578.441774,666,1200000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999997,1496051951404,95,6442,d,10000,21398,d,1,0.547826,19444.324786,5000,47000,0.0
8999998,1496051971598,96,5470,b,19000,21398,a,0,0.545455,19444.324786,5000,47000,0.0
9999998,1496051971598,96,5470,b,19000,21398,a,0,0.543103,19444.324786,5000,47000,1.0
8999999,1496052005208,97,5198,d,32000,21398,d,1,0.545064,19444.324786,5000,47000,1.0


In [17]:
merged_df.dropna(subset=['next_is_correct'], inplace=True)

In [18]:
merged_df

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id,correct_answer,is_correct,correctness_rate,average_elapsed_time,min_elapsed_time,max_elapsed_time,next_is_correct
0,1565096190868,1,5012,b,38000,1,c,0,0.000000,49578.441774,666,1200000,1.0
1,1565096221062,2,4706,c,24000,1,c,1,0.500000,49578.441774,666,1200000,1.0
2,1565096293432,3,4366,b,68000,1,b,1,0.666667,49578.441774,666,1200000,0.0
3,1565096339668,4,4829,a,42000,1,c,0,0.500000,49578.441774,666,1200000,0.0
4,1565096401774,5,6528,b,59000,1,d,0,0.400000,49578.441774,666,1200000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8999997,1496051951404,95,6442,d,10000,21398,d,1,0.545852,19444.324786,5000,47000,1.0
9999997,1496051951404,95,6442,d,10000,21398,d,1,0.547826,19444.324786,5000,47000,0.0
8999998,1496051971598,96,5470,b,19000,21398,a,0,0.545455,19444.324786,5000,47000,0.0
9999998,1496051971598,96,5470,b,19000,21398,a,0,0.543103,19444.324786,5000,47000,1.0


In [19]:
features = ['correctness_rate', 'average_elapsed_time', 'min_elapsed_time', 'max_elapsed_time']

In [20]:
from sklearn.model_selection import train_test_split

X = merged_df[features]
y = merged_df['next_is_correct']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [33]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.5775283333333333
Precision: 0.6824782686565699
Recall: 0.6915641563659145
F1-score: 0.6869911721886278


In [34]:
# Assuming you have a new_data dataframe with the same features as X_test
new_data_predictions = model.predict(merged_df[features])


In [36]:
pip install xgboost


In [37]:
from xgboost import XGBClassifier

# Initialize the XGBoost classifier
model = XGBClassifier(random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.6773616666666666
Precision: 0.68300807802619
Recall: 0.9679864557125668
F1-score: 0.800902189753996


In [38]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression classifier
model = LogisticRegression(random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.6703933333333333
Precision: 0.6703933333333333
Recall: 1.0
F1-score: 0.8026772137500549


In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

# Assuming you have the dataframe merged_df with features and target variable 'next_is_correct'

# Split the data into features (X) and target variable (y)
features = ['correctness_rate', 'average_elapsed_time', 'min_elapsed_time', 'max_elapsed_time']
X = merged_df[features].values
y = merged_df['next_is_correct'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features (optional but can improve training)
X_train = (X_train - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)
X_test = (X_test - np.mean(X_train, axis=0)) / np.std(X_train, axis=0)

# Initialize the neural network model
model = Sequential()

# Add layers to the model
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions on the testing data
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Epoch 1/10
60000/60000 [==============================] - 140s 2ms/step - loss: 0.6124 - accuracy: 0.6771 - val_loss: 0.6117 - val_accuracy: 0.6776
Epoch 2/10
60000/60000 [==============================] - 181s 3ms/step - loss: 0.6116 - accuracy: 0.6773 - val_loss: 0.6114 - val_accuracy: 0.6776
Epoch 3/10
60000/60000 [==============================] - 153s 3ms/step - loss: 0.6115 - accuracy: 0.6773 - val_loss: 0.6114 - val_accuracy: 0.6776
Epoch 4/10
60000/60000 [==============================] - 136s 2ms/step - loss: 0.6115 - accuracy: 0.6774 - val_loss: 0.6114 - val_accuracy: 0.6776
Epoch 5/10
60000/60000 [==============================] - 135s 2ms/step - loss: 0.6114 - accuracy: 0.6774 - val_loss: 0.6114 - val_accuracy: 0.6777
Epoch 6/10
60000/60000 [==============================] - 137s 2ms/step - loss: 0.6114 - accuracy: 0.6774 - val_loss: 0.6114 - val_accuracy: 0.6778
Epoch 7/10
60000/60000 [==============================] - 136s 2ms/step - loss: 0.6114 - accuracy: 0.6775 - val_

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Initialize the base classifier (in this case, a decision tree)
base_classifier = DecisionTreeClassifier(max_depth=3)

# Initialize the AdaBoost classifier
model = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=50, random_state=42)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.677604
Precision: 0.6834355997072005
Recall: 0.9651015479778974
F1-score: 0.8002061174621649
